In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd /content/drive/My\ Drive/tensorflow_object_counting_api

In [0]:
%load_ext autoreload
%autoreload 2

### Установка зависимостей

In [0]:
!pip install -r requirements.txt

In [0]:
!sudo apt-get install protobuf-compiler python-pil python-lxml python-tk

#### models

In [0]:
!git clone https://github.com/tensorflow/models.git

#### create python files

In [0]:
%cd /content/drive/My\ Drive/tensorflow_object_counting_api/models/research
!protoc object_detection/protos/*.proto --python_out=.

#### pycocotols

In [0]:
%cd /content/drive/My\ Drive/tensorflow_object_counting_api

!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
!cp -r pycocotools /content/drive/My\ Drive/tensorflow_object_counting_api/models/research/

#### PYTHONPATH сhange

In [0]:
! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/drive/My Drive/tensorflow_object_counting_api/models/research"
os.environ['PYTHONPATH'] += ":/content/drive/My Drive/tensorflow_object_counting_api/models/research/slim"
os.environ['PYTHONPATH'] += ":/content/drive/My Drive/tensorflow_object_counting_api/models/research/object_detection"
os.environ['PYTHONPATH'] += ":/content/drive/My Drive/tensorflow_object_counting_api"

! echo $PYTHONPATH

### Использование предобученной модели

Для использования предобученной модели загрузите ее в корневую папку проекта. Список моделей: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md .

Для запуска необходимо указать имя модели и название карты меток датасета, на котором обучалась модель, карты меток можно найти в папке data.

Пример: после загрузки, для модели faster_rcnn_nas_coco_2018_01_28 указываем 

detection_graph, category_index = backbone.set_model('faster_rcnn_nas_coco_2018_01_28', 'mscoco_label_map.pbtxt')

Используйте run

In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_nas_coco_2018_01_28.tar.gz

In [0]:
!tar -xvf faster_rcnn_nas_coco_2018_01_28.tar.gz

In [0]:
!python run/targeted_object_counting.py

### Cоздание собственного классификатора

1. Загрузите модель.

2. Загрузите наборы изображений и их xml-аннотации в соответствующие папки image_data/train, image_data/test.

3. Настройте пользовательские файлы generate_tfrecord.py, xml_to_csv.py в папке tools.

4. С помощью xml_to_csv.py создайте файлы аннотаций test_labels.csv, test_labels.csv, затем на их основе создайте файлы test.record, train.record c помощью generate_tfrecord.py. Все файлы должны находиться в папке image_data.

5. Скопируйте необходимый файл конфигурации в папку training, настройте его. Полный список конфигураций можно найти в списке https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs , либо в папке models/research/object_detection/samples. Также создайте собственную карту меток в папке data.

6. Запустите train.py

In [0]:
!python tools/generate_tfrecord.py --csv_input=image_data/train_labels.csv --image_dir=image_data/train --output_path=image_data/train.record

In [0]:
!python tools/generate_tfrecord.py --csv_input=image_data/test_labels.csv --image_dir=image_data/test --output_path=image_data/test.record

In [0]:
!python tools/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_nas_coco.config

#### Tensorboard

In [0]:
%tensorboard --logdir=training

#### Выгрузка модели из контрольной точки 

In [0]:
!python tools/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path training/faster_rcnn_nas_coco.config \
    --trained_checkpoint_prefix training/model.ckpt-0 \
    --output_directory /content/drive/My\ Drive/tensorflow_object_counting_api/my_classifier

#### Использование 

Для использования классификатора по аналогии с предобученной моделью укажите папку my_classifier и my_labelmap
Пример: detection_graph, category_index = backbone.set_model('my_classifier', 'my_labelmap.pbtxt')

Используйте run

In [0]:
!python run/targeted_object_counting.py